# Задание №1+ Дополнительная часть

In [1]:
"""Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>"""
import requests
from bs4 import BeautifulSoup
import time

In [2]:
"""Находим ссылки новостей"""

def get_all_links(URL):
    res = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('li', class_='content-list__item')
    article_post = soup.find_all('h2', 'post__title')
    all_refs = list(map(lambda x: x.find('a').get('href'),article_post))
   
    return all_refs

In [3]:
URL = 'https://habr.com/ru/all/'
all_refs = get_all_links (URL)
all_refs

['https://habr.com/ru/post/543844/',
 'https://habr.com/ru/post/541852/',
 'https://habr.com/ru/post/543602/',
 'https://habr.com/ru/post/543832/',
 'https://habr.com/ru/company/action360/blog/543804/',
 'https://habr.com/ru/post/543838/',
 'https://habr.com/ru/post/543790/',
 'https://habr.com/ru/post/543830/',
 'https://habr.com/ru/post/543834/',
 'https://habr.com/ru/post/543828/',
 'https://habr.com/ru/post/543700/',
 'https://habr.com/ru/company/timeweb/blog/543822/',
 'https://habr.com/ru/post/543818/',
 'https://habr.com/ru/company/itsoft/blog/543812/',
 'https://habr.com/ru/post/543806/',
 'https://habr.com/ru/post/543802/',
 'https://habr.com/ru/post/543798/',
 'https://habr.com/ru/post/543774/',
 'https://habr.com/ru/company/englishdom/blog/543720/',
 'https://habr.com/ru/company/skillfactory/blog/543788/']

In [4]:
import pandas as pd
import re

In [5]:
"""1 вариант"""
def get_kom_news(links,KEYWORDS):
    kom_news = pd.DataFrame()
    for link in all_refs:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        text = soup.find('div', class_='post__body post__body_full').text
        for word in KEYWORDS:
            """проходим циклом по искомым словам"""
            if word in text:
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()  
                title = soup.find('h1', class_='post__title post__title_full').text
                text_key = text
                link_key = link
                row = {'date': date, 'link': link_key,'title': title, 'text': text_key}
                kom_news = pd.concat([kom_news, pd.DataFrame([row])])
            else:
                continue                
    return kom_news

In [6]:
KEYWORDS = ['python','Python','парс']
kom_news = get_kom_news(all_refs, KEYWORDS)
kom_news

,date,link,title,text
0,2021-02-23,https://habr.com/ru/company/action360/blog/543...,"\nVR-тренинги, EdTech-стартапы и электронный д...",\nМы начали обсуждать трансформацию сферы обра...
0,2021-02-23,https://habr.com/ru/company/timeweb/blog/543822/,\nПолезные расширения VS Code для JavaScript-р...,\nНедавно мы опубликовали перевод статьи про п...
0,2021-02-23,https://habr.com/ru/post/543806/,\nАудит паролей Активной Директории Windows\n,\nЕсли в вашей компании активно используется д...
0,2021-02-23,https://habr.com/ru/post/543806/,\nАудит паролей Активной Директории Windows\n,\nЕсли в вашей компании активно используется д...
0,2021-02-23,https://habr.com/ru/post/543798/,\nРеализация Minecraft Query протокола в .Net ...,\nMinecraft Server Query – это простой протоко...
0,2021-02-23,https://habr.com/ru/company/skillfactory/blog/...,\nКибервойна. Когда 500 Кб кода страшнее межко...,"\n\r\nПривет, Хабр! Мы искренне поздравляем вс..."


In [7]:
# """2 вариант"""
# def get_kom_news_1 (links):
#     kom_news = pd.DataFrame()
#     for link in all_refs:
#         soup = BeautifulSoup(requests.get(link).text, 'html.parser')
#         if soup.find('span', class_='post__time'):
#             date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
#         if soup.find('h1', class_='post__title post__title_full'): 
#             title = soup.find('h1', class_='post__title post__title_full').text
#         text = soup.find('div', class_='post__body post__body_full').text
#         row = {'date': date, 'link': link,'title': title, 'text': text}
#         kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
#     return kom_news

In [8]:
# kom_news_2 = get_kom_news_1 (all_refs)
# kom_news_2.head()

In [9]:
"""Выдает ошибку, не поняла почему."""
# kom_news_2[kom_news_2.text.str.contains('Python',case = False)].head()

'Выдает ошибку, не поняла почему.'

# Задача №2

In [12]:
"""
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>"""
import json
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
emails = ['lismak1@mail.ru', 'salavat-m@mail.ru']
res_passwords = pd.DataFrame()
for email in emails:
    params = {'emailAddresses':[email]}
    time.sleep(0.33)
    req = requests.post(URL,  data = json.dumps(params), headers =  {'Vaar-Version': '0','Vaar-Header-App-Product': 'hackcheck-web-avast'})
    req_1 = pd.DataFrame(req.json()['breaches'])
    res_ = req_1.transpose()
    result = res_.loc[:,['publishDate','site','description','statistics']]
    result['Почта'] = email
    res_passwords = pd.concat([res_passwords, result])
res_passwords

,publishDate,site,description,statistics,Почта
12,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...,"{'usernames': 0, 'passwords': 107425364, 'emai...",lismak1@mail.ru
13460,2018-01-20T00:00:00Z,k-vrachu.ru,"In January 2016, K-Vrachu's database was alleg...","{'usernames': 463418, 'passwords': 463421, 'em...",lismak1@mail.ru
17609,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo...","{'usernames': 0, 'passwords': 0, 'emails': 205...",salavat-m@mail.ru
60,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro...","{'usernames': 0, 'passwords': 29231325, 'email...",salavat-m@mail.ru
4505,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM...","{'usernames': 6848208, 'passwords': 6852955, '...",salavat-m@mail.ru
12,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...,"{'usernames': 0, 'passwords': 107425364, 'emai...",salavat-m@mail.ru
13460,2018-01-20T00:00:00Z,k-vrachu.ru,"In January 2016, K-Vrachu's database was alleg...","{'usernames': 463418, 'passwords': 463421, 'em...",salavat-m@mail.ru
